In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas import read_csv
from numpy import set_printoptions

In [2]:
os.chdir('H:\Fake review detector\Feature-based-fake-review-detection-main')

In [3]:
data = pd.read_csv("amazon_reviews.txt", delimiter = "\t")

In [4]:
data.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [5]:
amazon_reviews = pd.read_csv("amazon_reviews.txt", delimiter = "\t")
df = amazon_reviews[['LABEL','REVIEW_TITLE', 'REVIEW_TEXT']]
df.rename(columns = {'LABEL': 'Label', 'REVIEW_TITLE': 'Review_Title', 'REVIEW_TEXT': 'Review_Text'}, inplace = True)

C:\Users\91781\AppData\Local\Temp\ipykernel_18748\3615045357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'LABEL': 'Label', 'REVIEW_TITLE': 'Review_Title', 'REVIEW_TEXT': 'Review_Text'}, inplace = True)


In [6]:
df['Review_Text'] = df['Review_Title'] + " " + df['Review_Text']

C:\Users\91781\AppData\Local\Temp\ipykernel_18748\433396366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Review_Text'] = df['Review_Title'] + " " + df['Review_Text']


In [7]:
df=df.drop(['Review_Title'], axis=1)

In [8]:
# Remove N/A
for col in df.columns:
    print(col, df[col].isnull().sum())
    df = df.dropna()

Label 0
Review_Text 0


In [9]:
import contractions

In [10]:
df['no_contract'] = df['Review_Text'].apply(lambda x: [contractions.fix(word) for word in x.split()]) 
df['REVIEW_TEXT_str'] = [' '.join(map(str, l)) for l in df['no_contract']]

In [11]:
from bs4 import BeautifulSoup

In [12]:
# Remove html tags
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

new_list = []
for rows in df['REVIEW_TEXT_str']:
    new_list.append(strip_html_tags(rows))
df['tag_removed'] = new_list

C:\Users\91781\AppData\Local\Temp\ipykernel_18748\350002440.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [13]:
df.head()

,Label,Review_Text,no_contract,REVIEW_TEXT_str,tag_removed
0,__label1__,"useful When least you think so, this product w...","[useful, When, least, you, think, so,, this, p...","useful When least you think so, this product w...","useful When least you think so, this product w..."
1,__label1__,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...",New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...
2,__label1__,doesn't swing very well. I purchased this swin...,"[does not, swing, very, well., I, purchased, t...",does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...
3,__label1__,Great computing! I was looking for an inexpens...,"[Great, computing!, I, was, looking, for, an, ...",Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...
4,__label1__,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...",Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...


In [14]:
import re
# Removing Numbers
df['number_removed'] = df['tag_removed'].apply(lambda x:re.sub(r'\d+', '', x))

In [15]:
df.head()

,Label,Review_Text,no_contract,REVIEW_TEXT_str,tag_removed,number_removed
0,__label1__,"useful When least you think so, this product w...","[useful, When, least, you, think, so,, this, p...","useful When least you think so, this product w...","useful When least you think so, this product w...","useful When least you think so, this product w..."
1,__label1__,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...",New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...
2,__label1__,doesn't swing very well. I purchased this swin...,"[does not, swing, very, well., I, purchased, t...",does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...
3,__label1__,Great computing! I was looking for an inexpens...,"[Great, computing!, I, was, looking, for, an, ...",Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...
4,__label1__,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...",Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...


In [16]:
#nlkt -- for tokenization and wordnet
import nltk
nltk.download('all')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

[nltk_data] Error loading all: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [17]:
# Tokenization
df['tokenized'] = df['number_removed'].apply(word_tokenize)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\91781/nltk_data'
    - 'C:\\Users\\91781\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'C:\\Users\\91781\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'C:\\Users\\91781\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\91781\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
# Conversion to lowercase
df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])

In [ ]:
# Parts of speech tagging

df['pos_tags'] = df['lower'].apply(nltk.tag.pos_tag)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

In [ ]:
df.head()

In [ ]:
df.loc[df["Label"] == "__label1__", "Label"] = 0
df.loc[df["Label"] == "__label2__", "Label"] = 1

In [ ]:
df.head()

In [ ]:
# Save the DataFrame to a JSON file
df.to_json('output.json', orient='records')